# Classification 

- learn model (귀납)
- apply model (연역)

## Evaluation Metrics (평가지표)
### Confusion Matrix

In [ ]:
# accuracy = (f11+f00) /(f10+f11+f01+f00)

# error = (f10+f01) /(f10+f11+f01+f00)

# 01. Decision Tree 